In [ ]:
#=======================================================================================================
# This python notebook will retrain incetion V3 Model, save the weights and make predictions
#=======================================================================================================

# The directory structure is as follows:
#  root
#  |
#  flowers
#  |
#  - train
#      - roses
#      - daisy
#  - valid
#      - roses
#      - daisy    

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir ='data/flowers/sample/train' #contains two classes cats and dogs
validation_data_dir ='data/flowers/sample/valid' #contains two classes cats and dogs

nb_train_samples = 10 #1274
nb_validation_samples = 50 #1274
nb_epoch = 2 # The Number of Epochs to Train over
b_size = 5 # The Batch Size

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


print('\n')
print('===========================')
print('Model Compilation Complete')
print('===========================')

In [ ]:
# Summarize the Model Parameters
model.summary()

In [ ]:
# Now specify the Directories for Training and Validation

train_data_dir ='data/flowers/train' #contains two classes cats and dogs
validation_data_dir ='data/flowers/valid' #contains two classes cats and dogs

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255)#,
 #       shear_range=0.2,
 #       zoom_range=0.2,
 #       horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=b_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical'
)

history = model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    samples_per_epoch=128, #128
    validation_data=validation_generator,
    nb_val_samples= nb_validation_samples) #1020

print('\n')
print('====================================================')
print('Training History Model Complete: \n')
print('====================================================')



In [ ]:
# At this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
from keras.optimizers import SGD

N_freeze = 249

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first N_freeze layers and unfreeze the rest:
for layer in model.layers[:N_freeze]:
   layer.trainable = False
for layer in model.layers[N_freeze:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# we need to recompile the model for these modifications to take effect

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)
# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)


print('\n')
print('====================================================')
print('Training Complete: \n')
print('====================================================')

In [ ]:
# Save the model to an output file for use later

model.save("inception_retrained_model1")
##################################

print('\n')
print('====================================================')
print('Saving The Model is Complete: \n')
print('====================================================')

In [ ]:
# Load the re-trained model
from keras.models import load_model

retrained_model = load_model('inception_retrained_model1')

print('\n')
print('====================================================')
print('Loaded in the Retrained Model \n')
print('====================================================')

In [ ]:
from keras.preprocessing import image
import numpy as np
from keras.applications.resnet50 import preprocess_input, decode_predictions

# Let us use the inception v3 Model
img_path = 'roses_1.jpg'
#img_path = 'Elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))


x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = retrained_model .predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print(preds)